<a href="https://colab.research.google.com/github/tharunsaij/Fmml-Assignments/blob/main/Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [28]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [29]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-29-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [30]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [31]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [32]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [33]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [34]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [35]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [36]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [37]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [38]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [39]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
Increasing the percentage of the validation set provides a more reliable estimate of a model's performance on unseen data, making it useful for detecting overfitting. However, it reduces the amount of data available for training, potentially leading to underfitting.

Decreasing the percentage of the validation set allocates more data for training, aiding model learning, but may yield less reliable performance estimates and make overfitting harder to detect. The choice depends on factors like available data and computational resources. Techniques like k-fold cross-validation help balance these considerations for a robust evaluation.

2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
The size of the training and validation sets plays a crucial role in predicting a model's accuracy on the test set using the validation set:

- A large training set can lead to better model performance during training, but a small validation set may not reliably predict test set accuracy.
- A small training set may result in underfitting, while a large validation set offers a more robust performance estimate.
- A balanced split with reasonably sized train and validation sets often strikes a good compromise.
- Consider factors like dataset size, problem complexity, and overfitting risk when choosing the split size, and techniques like k-fold cross-validation can enhance performance prediction reliability.
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?
Certainly, here's a more concise version:

The percentage you reserve for the validation set should strike a balance between having enough data for training and for validation. Common splits are 70% for training, 15% for validation, and 15% for testing, but adjust based on your dataset size. For larger datasets, you can allocate less to validation and testing, and for smaller datasets, allocate more to ensure meaningful evaluation. Techniques like k-fold cross-validation can also help when data is limited.

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [40]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [41]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
Certainly! In summary, averaging validation accuracy across multiple data splits, known as cross-validation, provides more consistent and reliable results. This approach helps reduce the impact of random data splitting and offers a robust evaluation of a model's generalization performance, especially when working with limited datasets. Cross-validation methods like k-fold or stratified k-fold are commonly used for this purpose. It's a valuable technique for assessing model consistency and estimating its likely performance on new, unseen data.
2. Does it give more accurate estimate of test accuracy?
The accuracy of a test estimate depends on various factors, including the quality and quantity of data, the complexity of the model, and the evaluation metrics used. In machine learning, cross-validation techniques such as k-fold cross-validation can provide a more accurate estimate of test accuracy compared to a single train-test split. By splitting the data into multiple subsets and training/testing the model on different combinations, cross-validation can provide a more robust estimate of a model's performance.

Additionally, using appropriate evaluation metrics, like precision, recall, F1-score, or ROC AUC, can provide a more informative assessment of a model's performance, especially when dealing with imbalanced datasets or specific use cases.

Ultimately, the accuracy of the estimate depends on how well the chosen techniques and metrics align with the specific problem and dataset you're working with.
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
The effect of the number of iterations on an estimate depends on the specific algorithm or process being used. In many cases, increasing the number of iterations can lead to a more accurate estimate, but there are diminishing returns. Here are a few general points to consider:

1. **Accuracy**: In iterative algorithms like numerical integration or optimization, more iterations often lead to a more accurate result. This is because each iteration refines the estimate, reducing errors.

2. **Convergence**: Some algorithms converge to a solution, meaning that as you increase iterations, the estimate gets closer and closer to the true value. However, there's usually a point where further iterations don't significantly improve accuracy.

3. **Computational Cost**: More iterations typically require more computational resources and time. There's a trade-off between accuracy and computational cost.

4. **Convergence Rate**: The speed at which an algorithm converges can vary. Some algorithms converge quickly at the beginning and then slow down, while others have a more consistent convergence rate.

5. **Stopping Criteria**: Many iterative algorithms use stopping criteria to determine when to halt iterations. These criteria are often based on the convergence behavior and desired accuracy. For example, the algorithm may stop when the change between consecutive iterations falls below a certain threshold.

In summary, increasing the number of iterations can generally improve estimates, but it's essential to consider the trade-offs between accuracy and computational cost. The specific behavior will depend on the algorithm and problem you're dealing with.
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?
Increasing the number of iterations can help to some extent when dealing with a very small training dataset, as it allows the model to see the limited data more times and potentially learn better. However, there are limitations to this approach:

1. **Overfitting:** With very few examples, the model may start memorizing the training data instead of generalizing from it. Increasing iterations could exacerbate overfitting.

2. **Data Quality:** More iterations won't help if the training data is of low quality or unrepresentative of the problem.

3. **Computational Resources:** Increasing iterations can be computationally expensive, and there might be practical limits to how many iterations you can run.

4. **Hyperparameter Tuning:** You may need to fine-tune other hyperparameters like learning rate, batch size, or regularization techniques when working with a small dataset.

5. **Data Augmentation:** Data augmentation techniques can artificially increase the effective size of your training dataset by creating new examples from existing ones. This can be especially helpful when you have a small dataset.

6. **Transfer Learning:** If applicable, consider using pre-trained models or transfer learning techniques. They leverage knowledge from models trained on large datasets and can be fine-tuned on your small dataset.

In summary, while increasing iterations can be a part of the solution, it's essential to consider the broader context of your data and problem. Combining it with other strategies like data augmentation, hyperparameter tuning, and transfer learning can often yield better results with a small dataset.